# Fine-tuning a Model for Masked Language Modeling (MLM) Exam

In this exam, you will be tasked with performing dataset preprocessing and fine-tuning a model for a masked language modeling task. Complete each step carefully according to the instructions provided.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `bert-base-uncased` for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/math_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [1]:
! pip install datasets

In [2]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [3]:
dataset = load_dataset("CUTD/math_df", split='train')
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
test_dataset = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text'],
        num_rows: 2000
    })
})

In [5]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'text'],
    num_rows: 8000
})

In [6]:
test_dataset

Dataset({
    features: ['Unnamed: 0', 'text'],
    num_rows: 2000
})

## Step 2: Load the Pretrained Model and Tokenizer

Use a pre-trained model and tokenizer for this task. Initialize both in this step.

In [7]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initial

## Step 3: Preprocess the Dataset

Define a preprocessing function that tokenizes the text data and prepares the inputs for the model. Ensure that you truncate the sequences to a maximum length of 512 tokens and pad them appropriately.

**Bonus**: If you performed more comprehensive preprocessing, such as removing links, converting text to lowercase, or applying additional preprocessing techniques.

In [8]:
def preprocess_data(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs


train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
train_dataset = train_dataset.remove_columns(['Unnamed: 0', 'text'])
test_dataset = test_dataset.remove_columns(['Unnamed: 0', 'text'])

## Step 4: Define Training Arguments

Set up the training configuration, including parameters like learning rate, batch size, number of epochs, and weight decay.

In [10]:
from transformers import TFAutoModelForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Step 5: Initialize the Trainer

Initialize the Trainer using the model, training arguments, and datasets (both training and evaluation).

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

## Step 6: Fine-tune the Model

Run the training process using the initialized Trainer to fine-tune the model on the masked language modeling task.

In [12]:
trainer.train()

Step,Training Loss
500,0.339900
1000,0.000300


TrainOutput(global_step=1000, training_loss=0.17007171422243117, metrics={'train_runtime': 973.8338, 'train_samples_per_second': 8.215, 'train_steps_per_second': 1.027, 'total_flos': 2105638502400000.0, 'train_loss': 0.17007171422243117, 'epoch': 1.0})

In [13]:
trainer.evaluate()

{'eval_loss': 0.00017983134603127837,
 'eval_runtime': 74.7274,
 'eval_samples_per_second': 26.764,
 'eval_steps_per_second': 3.345,
 'epoch': 1.0}

In [14]:
model.save_pretrained("./fill_finetuned_model")
tokenizer.save_pretrained("./fill_finetuned_model")

('./fill_finetuned_model/tokenizer_config.json',
 './fill_finetuned_model/special_tokens_map.json',
 './fill_finetuned_model/vocab.txt',
 './fill_finetuned_model/added_tokens.json',
 './fill_finetuned_model/tokenizer.json')

## Step 7: Inference

Use the fine-tuned model for inference. Create a pipeline for masked language modeling and test it with a sample sentence.

In [17]:
from transformers import pipeline
fill = pipeline("fill-mask", model="/content/fill_finetuned_model", tokenizer="/content/fill_finetuned_model")
sentence = "A software engineer who disagrees with the established computer scientist's [MASK]"
result = fill(sentence)
print(f"Prediction: {result}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Prediction: [{'score': 0.9249717593193054, 'token': 0, 'token_str': '[PAD]', 'sequence': "a software engineer who disagrees with the established computer scientist's"}, {'score': 0.003570623230189085, 'token': 10740, 'token_str': 'opinions', 'sequence': "a software engineer who disagrees with the established computer scientist's opinions"}, {'score': 0.0026841459330171347, 'token': 15306, 'token_str': 'conclusions', 'sequence': "a software engineer who disagrees with the established computer scientist's conclusions"}, {'score': 0.002680051140487194, 'token': 5448, 'token_str': 'opinion', 'sequence': "a software engineer who disagrees with the established computer scientist's opinion"}, {'score': 0.001795353600755334, 'token': 6567, 'token_str': 'decisions', 'sequence': "a software engineer who disagrees with the established computer scientist's decisions"}]


In [18]:
sentence1 = "An entrepreneur who is developing a mobile app that utilizes satellite imagery to track population density for [MASK]"
result = fill(sentence1)
print(f"Prediction: {result}")

Prediction: [{'score': 0.9546941518783569, 'token': 0, 'token_str': '[PAD]', 'sequence': 'an entrepreneur who is developing a mobile app that utilizes satellite imagery to track population density for'}, {'score': 0.003010051790624857, 'token': 5109, 'token_str': 'decades', 'sequence': 'an entrepreneur who is developing a mobile app that utilizes satellite imagery to track population density for decades'}, {'score': 0.0026829103007912636, 'token': 2086, 'token_str': 'years', 'sequence': 'an entrepreneur who is developing a mobile app that utilizes satellite imagery to track population density for years'}, {'score': 0.0022705639712512493, 'token': 15106, 'token_str': 'convenience', 'sequence': 'an entrepreneur who is developing a mobile app that utilizes satellite imagery to track population density for convenience'}, {'score': 0.0020715382415801287, 'token': 27620, 'token_str': 'millennia', 'sequence': 'an entrepreneur who is developing a mobile app that utilizes satellite imagery to t